## graphcol_2
* the graph of `graphcol_1` is 9-colorable, it would be interesting to know that 8 or less color is enough or not.
* for this purpose, we'll develop (actually i wrote it for a [codesignal](https://codesignal.com) interview question...) a simple function in julia, that can be used to compute the chromatic number for *very small* graphs: `graphcol_bt`	


In [3]:
include("../shared/tomdcode.jl")
_JLAB_=true

true

In [15]:
md"""### The `graphcol_2` module
* the only member is the `graphcol_bt` function, it tries to color the graph using *at most* `maxcol` colors - in case of failure returns `-1` as the used number of colors.
"""|>display
tomdcode("src/graphcol_2.jl")

### The `graphcol_2` module

  * the only member is the `graphcol_bt` function, it tries to color the graph using *at most* `maxcol` colors - in case of failure returns `-1` as the used number of colors.


```julia
module graphcol_2
  using Graphs
  # the backtracking solution
  # it is a naive implementation w/o any smartness,
  # just administration

  function graphcol_bt(G::Vector{Vector{Int}}, maxcol::Int) # max number of colors
    # the actual color are in 1..maxcol
    N = length(G)

    forbidden = fill(0, maxcol, N)
    # colors currently forbidden for a particular node
    # forbidden=already reserved by some of its neighbour
    # reserved if >0

    # actual and returned colorings
    color = fill(0, N) # for work with
    color_ret = fill(0, N)

    # modifies the forbidden and color arrays
    function paint(node, c)
      oldc = color[node]
      if oldc > 0
        for t in G[node]
          forbidden[oldc, t] -= 1
        end
      end

      color[node] = c
      (c == 0) && return

      for t in G[node]
        forbidden[c, t] += 1
      end
    end

    found = false
    paint(1, 1)

    function trav(node)
      if node > N
        found = true
        color_ret .= color
        return
      end


      for c = 1:maxcol
        (forbidden[c, node] > 0) && continue
        paint(node, c)
        trav(node + 1)
        found && break
      end

      paint(node, 0) # restore the original state
    end # of trav

    trav(2)
    (found, color_ret)
  end


  # a method (variant) that takes a Graph() instance and returns a similar
  # object that is returned by Graphs.random_greedy_color
  # (imitating by namedtuple)
  function graphcol_bt(G::Graph, maxcol::Int) # max number of colors
    GG = [Int[] for n = 1:nv(G)]
    for e in edges(G)
      a, b = src(e), dst(e)
      push!(GG[a], b)
      push!(GG[b], a)
    end
    found, color = graphcol_bt(GG, maxcol)
    if found
      (num_colors = length(Set(color)), colors = color)
    else
      (num_colors = -1, colors = nothing)
    end
  end

  export graphcol_bt
end

```


In [4]:
md"""### The client code
* using `graphcol_bt` for small sized cyclic graphs (we know their chromatics)
* then, execute it for the "courses" graph of `graphcol_1` with `n=8,9` colors
"""|>display
tomdcode("main.jl")

### The client code

  * using `graphcol_bt` for small sized cyclic graphs (we know their chromatics)
  * then, execute it for the "courses" graph of `graphcol_1` with `n=8,9` colors


```julia
include("todev.jl"); todev(["../graphcol_1/src"])

using graphcol_1
using graphcol_2

printstyled("some small tests w/ cycle graphs\n",color=:white)
for t in 1:2
  n = rand(3:9)
  G2 = cycle_graph(n)
  printstyled("cycle, n=$(n)\n",color=:yellow)
  printstyled("\nusing 3 colors:\n",color=:green)
  @time a = graphcol_bt(G2, 3)
  println("  ",a)
  printstyled("\nusing 2 colors:\n",color=:green)
  @time b = graphcol_bt(G2, 2)
  println(b)
end

# and use graphcol_bt for the original data of project_1
data = graphcol_1_data()
G = data.G
printstyled("\norginal graph\n",color=:yellow)
printstyled("using 9 colors:\n",color=:green)
@time ok = graphcol_bt(G, 9)
println(ok)
printstyled("using 8 colors:\n",color=:green)
@time nok = graphcol_bt(G, 8)
println(nok)

```


In [11]:
md"""### The output
"""|>display
include("main.jl")

### The output


some small tests w/ cyclic graphs

vertices => 3

using 3 colors:
  0.000005 seconds (19 allocations: 1.359 KiB)
  (num_colors = 3, colors = [1, 2, 3])

using 2 colors:
  0.000002 seconds (14 allocations: 944 bytes)
(num_colors = -1, colors = nothing)

vertices => 4

using 3 colors:
  0.000002 seconds (21 allocations: 1.562 KiB)
  (num_colors = 2, colors = [1, 2, 1, 2])

using 2 colors:
  0.000002 seconds (21 allocations: 1.531 KiB)
(num_colors = 2, colors = [1, 2, 1, 2])


the "courses" graph

using 9 colors:
  0.000044 seconds (132 allocations: 50.570 KiB)
(num_colors = 11, colors = [1, 2, 3, 4, 5, 6, 7, 8, 1, 3, 2, 4, 5, 6, 7, 9, 8, 4, 1, 2, 3, 7, 5, 8, 9, 6, 2, 4, 9, 3, 7, 10, 11, 8, 1, 5, 6, 7, 2, 4])

using 8 colors:
  0.000049 seconds (132 allocations: 50.258 KiB)
(num_colors = 11, colors = [1, 2, 3, 4, 5, 6, 7, 8, 1, 3, 2, 4, 5, 6, 7, 9, 8, 4, 1, 2, 3, 7, 5, 8, 9, 6, 2, 4, 9, 3, 7, 10, 11, 8, 1, 5, 6, 7, 2, 4])


### Conclusion
* even for this small graph this backtracking solution is very slow, but after executing it we can be sure that fewer than 9 colors (exam dates) is not enough.